In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score

# === 1) Načtení obrazu ===
img = cv2.imread("pvi_cv08_sunflowers1.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# === 2) Laplacian of Gaussian (LoG) ===
# Detekce blobů – tmavé oblasti na světlém pozadí
log = cv2.Laplacian(gray, cv2.CV_64F, ksize=5)
log = cv2.convertScaleAbs(log)

# Inverze (protože detekujeme tmavé bloby)
log_inv = cv2.bitwise_not(log)

# === 3) Prahování LoG výsledku ===
_, binary = cv2.threshold(log_inv, 180, 255, cv2.THRESH_BINARY)

# === 4) Najdi potenciální blob oblasti ===
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)

# Volitelná sigma pro velikost blobu
sigma = 10
r = int(2 * sigma)  # polovina čtverce (strana 4*sigma)

# === 5) Vizualizace detekovaných blobů ===
vis = img.copy()
for i, c in enumerate(centroids[1:], start=1):  # přeskočíme pozadí
    x, y = int(c[0]), int(c[1])
    cv2.circle(vis, (x, y), 6, (0, 0, 255), 2)
    cv2.rectangle(vis, (x - r, y - r), (x + r, y + r), (255, 0, 0), 2)

# === 6) Vyhodnocení – precision a recall ===
# Ukázková data ground truth (nahraď vlastními body!)
gt_points = np.array([[200, 300], [500, 250], [800, 400]])  # příklad
pred_points = np.array(centroids[1:])

# Funkce pro porovnání (na základě vzdálenosti)
def match_points(gt, pred, tol=25):
    y_true, y_pred = [], []
    for gx, gy in gt:
        found = False
        for px, py in pred:
            if np.hypot(px - gx, py - gy) < tol:
                found = True
                break
        y_true.append(1)
        y_pred.append(1 if found else 0)
    return precision_score(y_true, y_pred), recall_score(y_true, y_pred)

precision, recall = match_points(gt_points, pred_points)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

# === 7) Zobrazení výsledků ===
plt.figure(figsize=(15, 7))
plt.subplot(1, 3, 1)
plt.title("Gray Image")
plt.imshow(gray, cmap="gray")

plt.subplot(1, 3, 2)
plt.title("LoG Filtered")
plt.imshow(log_inv, cmap="gray")

plt.subplot(1, 3, 3)
plt.title("Detected Sunflowers (LoG)")
plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
import cv2, numpy as np, matplotlib.pyplot as plt
from scipy.stats import entropy
from pathlib import Path
import glob

SIGMA = 8
HIST_BINS = 32
DIVERGENCE_THRESHOLD = 1.0   # larger = looser filtering

# --- helpers -------------------------------------------------------------
def hue_hist_from_hsv(hsv, mask=None):
    h = hsv[..., 0]
    hist = cv2.calcHist([h], [0], mask, [HIST_BINS], [0, 180])
    hist = hist.flatten().astype(np.float64)
    hist /= (hist.sum() + 1e-9)
    return hist

def IoU(a, b):
    xA, yA = max(a[0], b[0]), max(a[1], b[1])
    xB, yB = min(a[2], b[2]), min(a[3], b[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    if inter == 0: return 0
    areaA = (a[2]-a[0])*(a[3]-a[1])
    areaB = (b[2]-b[0])*(b[3]-b[1])
    return inter / (areaA + areaB - inter)

def load_gt(txtfile):
    g=[]
    with open(txtfile) as f:
        for line in f:
            vals=line.strip().split()
            if len(vals)==4:
                g.append(list(map(int,vals)))
    return g

# --- adaptive template histogram ----------------------------------------
def adaptive_template_hist(template):
    hsv = cv2.cvtColor(template, cv2.COLOR_BGR2HSV)
    dark_mask = (hsv[...,2] < 130) & (hsv[...,1] > 60)
    if dark_mask.sum() < 50:
        return hue_hist_from_hsv(hsv)
    return hue_hist_from_hsv(hsv, dark_mask.astype(np.uint8))

# ------------------------------------------------------------------------
template = cv2.imread("pvi_cv08_sunflower_template.jpg")
template_hist = adaptive_template_hist(template)

# gather images
images = sorted(glob.glob("pvi_cv08_sunflowers*.jpg"))
precisions, recalls = [], []

for img_path in images:
    txt_path = Path(img_path).with_suffix(".txt")
    img = cv2.imread(img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gt_boxes = load_gt(txt_path)

    # --- LoG detection ---
    gray_blur = cv2.GaussianBlur(gray, (0,0), SIGMA)
    log = cv2.Laplacian(gray_blur, cv2.CV_64F)
    log = cv2.convertScaleAbs(log)
    log_inv = cv2.bitwise_not(log)
    _, binary = cv2.threshold(log_inv, 180, 255, cv2.THRESH_BINARY)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE,
                              cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(9,9)))

    nlab, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
    detected=[]
    for i in range(1,nlab):
        x,y,w,h,area = stats[i]
        if area < 120: continue
        cx,cy = int(centroids[i][0]), int(centroids[i][1])
        side=int(4*SIGMA)
        x1,y1=max(0,cx-side//2),max(0,cy-side//2)
        x2,y2=min(img.shape[1],cx+side//2),min(img.shape[0],cy+side//2)
        patch = hsv[y1:y2, x1:x2]
        if patch.size==0: continue
        h_hist = hue_hist_from_hsv(patch)
        kl = entropy(h_hist + 1e-3, template_hist + 1e-3)
        if kl < DIVERGENCE_THRESHOLD:
            detected.append([x1,y1,x2,y2])

    # --- Evaluation ---
    TP=sum(any(IoU(gt,d)>=0.5 for d in detected) for gt in gt_boxes)
    FP=len(detected)-TP
    FN=len(gt_boxes)-TP
    prec = TP/(TP+FP+1e-6)
    rec  = TP/(TP+FN+1e-6)
    precisions.append(prec); recalls.append(rec)
    print(f"{Path(img_path).name}:  TP={TP}, FP={FP}, FN={FN}  → P={prec:.3f}, R={rec:.3f}")

    # --- Visualization ---
    vis=img.copy()
    for (x1,y1,x2,y2) in detected:
        cv2.rectangle(vis,(x1,y1),(x2,y2),(255,0,255),2)
    for (x1,y1,x2,y2) in gt_boxes:
        cv2.rectangle(vis,(x1,y1),(x2,y2),(255,255,255),1)

    plt.figure(figsize=(8,6))
    plt.title(f"{Path(img_path).name}  Prec={prec:.2f}, Rec={rec:.2f}")
    plt.imshow(cv2.cvtColor(vis,cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    plt.figure(figsize=(10,8))
    plt.title("LoG response")
    plt.imshow(log_inv, cmap="gray")
    plt.show()

    plt.figure(figsize=(10,8))
    plt.title("Binary mask after threshold")
    plt.imshow(binary, cmap="gray")
    plt.show()


print(f"\nMean Precision={np.mean(precisions):.3f},  Mean Recall={np.mean(recalls):.3f}")
